# Final Term - Exercise 1
### David Della Rossa


In [30]:
import sys
!{sys.executable} -m pip install opencv-python

import cv2

### Task 1

In [31]:
# Create a VideoCapture object and read the frames from an input file
video=cv2.VideoCapture('Exercise1_Files\Traffic_Laramie_1.mp4')

# Check if the video opened successfully
if not video.isOpened(): 
    print("Error opening video file")
    exit(0)

#create an instance of type K-nearest neighbours background subtraction
backSub = cv2.createBackgroundSubtractorKNN(detectShadows = False)

# # Read until video is completed or we press 'q'
while True:
    # Capture frame-by-frame
    # Note that VideoCapture captures the frames of a video without considering the fps of the video
    check, frame = video.read()
    
    if check == False:
        break;
        
    #calculate the foreground mask - this is the frame difference between the original frame and the calculated background
    fgMask = backSub.apply(frame)
    
    # Dilate and erode to get object blobs
    fgMask = cv2.dilate(fgMask, None, 18)
    fgMask = cv2.erode(fgMask, None, 10)
    
    # The difference (the delta_frame) is converted into a binary image
    # If a particular pixel value is greater than a certain threshold (specified by us here as 150),
    # it will be assigned the value for White (255) else Black(0)
    # Important: you may have to change the threshold value for a better performance with your webcam , room's light, etc.
    threshold_frame=cv2.threshold(fgMask, 100, 255, cv2.THRESH_BINARY)[1]
    
    

    # The cv2.findContours() method we will identify all the contours in our image.
    # This method expects 3 parameters, (a) image, (b) contour retrieval mode and
    # (c) contour approximation method
    (contours,_)=cv2.findContours(threshold_frame,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)

    
    for c in contours:
        # contourArea() method filters out any small contours
        (x, y, w, h)=cv2.boundingRect(c)
        #filter out contours with a form factor that identifies human shapes
        if (h/w > 1.5)  | (cv2.contourArea(c) < 500):
            continue
        #draw a rectangle around the bounding box
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0,255,0), 1)
    

    #Show frames on screen
    if check == True:
        cv2.imshow('Webcam', frame)  
        cv2.imshow('Threshold frame', threshold_frame)
        cv2.imshow('FG Mask', fgMask)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    
    else:
        break

# After the loop release the video object
video.release()

# Destroy all the windows
cv2.destroyAllWindows()

### Task 2

In [32]:
from datetime import datetime

In [33]:
# Create a VideoCapture object and read the frames from an input file
video=cv2.VideoCapture('Exercise1_Files\Traffic_Laramie_1.mp4')

# Capture the frame per seconds from the video. Used for calculating a precise number of cars per seconds
fps = video.get(cv2.CAP_PROP_FPS)
    
# Check if the video opened successfully
if not video.isOpened(): 
    print("Error opening video file")
    exit(0)


#create an instance of type K-nearest neighbours background subtraction
backSub = cv2.createBackgroundSubtractorKNN(detectShadows = False)

#stores Bounding boxes from previous frame
previousBboxes = []

#values used to correlate bounding boxes from previous frame to current frame
#on X axis
deltaPosX = 10
#on Y axis
deltaPosY = 10
#on horizontal size
deltaSizeH = 10
#on vertical size
deltaSizeW = 10

#threshold that determines when a bounding box moving left is to count
xThreshold = 200
#threshold that determines how many frames the bounding box has to be tracked along before counting
numOfFramesThreshold = 10

#number of cars per minute going towards the city centre
carsPerMinute = 0

startTime = datetime.now()

#number of vehicles counted
counter = 0

#Keep information about movements of bounding boxes
class bBoxMov:
    def __init__(self, x, y, w, h):
        self.x = x
        self.y = y
        self.w = w
        self.h = h
        #direction of movement
        self.dir = ''
        #whether the box has already been counted
        self.counted = False
        #number of frames the bounding box has been tracked along
        self.numOfFrames = 0

        
# Read until video is completed or we press 'q'
while True:
    # Capture frame-by-frame
    # Note that VideoCapture captures the frames of a video without considering the fps of the video
    check, frame = video.read()
    
    if check == False:
        break;
        
    #calculate the foreground mask - this is the frame difference between the original frame and the calculated background
    fgMask = backSub.apply(frame)
    
    # Dilate and erode to get object blobs
    fgMask = cv2.dilate(fgMask, None, 18)
    fgMask = cv2.erode(fgMask, None, 10)
    
    # The difference (the delta_frame) is converted into a binary image
    # If a particular pixel value is greater than a certain threshold (specified by us here as 150),
    # it will be assigned the value for White (255) else Black(0)
    # Important: you may have to change the threshold value for a better performance with your webcam , room's light, etc.
    threshold_frame=cv2.threshold(fgMask,100,255, cv2.THRESH_BINARY)[1]
    
    
    # The cv2.findContours() method we will identify all the contours in our image.
    # This method expects 3 parameters, (a) image, (b) contour retrieval mode and
    # (c) contour approximation method
    (contours,_)=cv2.findContours(threshold_frame,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)

    #bounding boxes identified in the current frame
    currentBboxes = []
    
    for c in contours:
        # contourArea() method filters out any small contours
        (x, y, w, h)=cv2.boundingRect(c)
        #filter out contours with a form factor that identifies human shapes
        if (h/w > 1.5)  | (cv2.contourArea(c) < 500):
            continue
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0,255,0), 1)
        #Create a new instance for the bounding box and add it to the collection
        currentBboxes.append(bBoxMov(x, y, w, h))
    
    #foreach BBoxes in currentBBoxes
    #check whether it is found in previousBBoxes
    #if yes, calculate the movement direction and store it
    #if not, it is a new BBox.
    #if a BBox goes out of screen on the left, increase counter.
    
    for cBbox in currentBboxes:
        found = False 
        
        for pBbox in previousBboxes:
            
            #distance between bounding boxes from previous to current frame
            deltaX = cBbox.x - pBbox.x
            deltaY = cBbox.y - pBbox.y
            
            #difference in size from previous to current frame
            deltaW = cBbox.w - pBbox.w
            deltaH = cBbox.h - pBbox.h
            
            #if this condition comes false, the current bbox is too far from previous box and is considered different
            if (abs(deltaX) > deltaPosX) | (abs(deltaY) > deltaPosY) \
                | (abs(deltaW) > deltaSizeW) |  (abs(deltaH) > deltaSizeH):
                continue
                
            #if we are here, it is a match
            found = True
            
            #label the box, showing the direction and the number of frames
            cv2.putText(frame, f'dir:{pBbox.dir}, frames:{pBbox.numOfFrames}', 
                        (cBbox.x, cBbox.y), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0), 1, cv2.LINE_AA)
                       

            #calculate direction on X
            if deltaX > 0:
                cBbox.dir = 'right'
            elif deltaX < 0:
                cBbox.dir = 'left'
                cBbox.numOfFrames = pBbox.numOfFrames + 1
            else:
                cBbox.dir = ''

            #keeps track of counted bboxes from frame to frame    
            cBbox.counted = pBbox.counted
                
                
            #going out of frame 
            if (cBbox.x < xThreshold) and (cBbox.numOfFrames > numOfFramesThreshold) \
                and (cBbox.dir == 'left') and (cBbox.counted == False):
                #this is a match
                counter+=1
                cBbox.counted = True
                
        #if previous bbox found, skip the remaining        
        if found == True:
            continue

    #delta time from the beginning of the video up to the current frame
    deltaTime = datetime.now() - startTime 
    #number of seconds from the beginning
    secs = deltaTime.total_seconds()
    #number of cars detected per minute
    carsPerMinute = counter * 60 / secs
    
    #Draw information on screen
    cv2.putText(frame, f'Count:{counter} in {int(secs)}s - ({carsPerMinute:.2f} cars/min)', (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 2, cv2.LINE_AA)
        
    
    #assign current Bboxes to previous
    previousBboxes = currentBboxes

    
    #Show frames on screen
    if check == True:
        cv2.imshow('Webcam', frame)  
        
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    
    else:
        break

print(f'Total number of cars: {counter}')
print(f'Cars per minute: {carsPerMinute:.2f}')

# After the loop release the video object
video.release()

# Destroy all the windows
cv2.destroyAllWindows()

Total number of cars: 6
Cars per minute: 8.79
